In [1]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers

You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Serverless-Whisper-ASR\venv\Scripts\python.exe -m pip install --upgrade pip' command.


  Running command git clone --filter=blob:none -q https://github.com/huggingface/transformers 'C:\Users\ayber\AppData\Local\Temp\pip-req-build-jnxrxfyr'
You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Serverless-Whisper-ASR\venv\Scripts\python.exe -m pip install --upgrade pip' command.



  Cloning https://github.com/huggingface/transformers to c:\users\ayber\appdata\local\temp\pip-req-build-jnxrxfyr
  Resolved https://github.com/huggingface/transformers to commit 9e56aff58a742b48fc8edea8d28d5b80330efbcc
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached regex-2022.10.31-cp39-cp39-win_amd64.whl (267 kB)
  Using cached tokenizers-0.13.2-cp39-cp39-win_amd64.whl (3.3 MB)
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5945973 sha256=4960d0e44ab891896f7327a8e949b4a6953f968fb55524b6110f22c117adef4b
  Stored in directory: C:\Users\ayber\AppData\Local\Temp\pip-ephem-wheel-cache-gln2sgi5\wheels\14\a0\7b\8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b66191909

In [2]:
!pip install evaluate>=0.30
!pip install jiwer

You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Serverless-Whisper-ASR\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Serverless-Whisper-ASR\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from datasets import DatasetDict

common_voice = DatasetDict.load_from_disk("gdrive/MyDrive/scalableML/Lab2/Dataset/common_voice")

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

def model_init(trial):
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
    model.config.forced_decoder_ids = None
    model.config.suppress_tokens = []
    return model

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    num_train_epochs=1,
    output_dir="whisper-small-se",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-4,
    warmup_steps=150,
    max_steps=300,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=300,
    eval_steps=300,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    hub_strategy="checkpoint",
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=None,
    model_init=model_init,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
!pip install wandb

In [ ]:
def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "objective", "goal": "minimize"},
        "parameters": {
            "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1e-3},
            "warmup_steps": {"distribution": "uniform", "min": 0, "max": 400},
            "weight_decay ": {"distribution": "uniform", "min": 0, "max": 0.01}
        },
    }

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="minimize",
    backend="wandb",
    hp_space=wandb_hp_space,
    n_trials=4,
)

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. 'weight_decay ' does not match '^^[A-Za-z_][A-Za-z0-9_.-]*$'
wandb sweep id - 4q6ikbs9


Create sweep with ID: 4q6ikbs9
Sweep URL: https://wandb.ai/ayberk/uncategorized/sweeps/4q6ikbs9


wandb: Agent Starting Run: kquhxk3n with config:
wandb: 	learning_rate: 0.0003267958009868873
wandb: 	warmup_steps: 346.1570450585282
wandb: 	weight_decay : 0.0024842683589863003
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Trying to set weight_decay  in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
W&B Sweep parameters: {'learning_rate': 0.0003267958009868873, 'warmup_steps': 346.1570450585282, 'weight_decay ': 0.0024842683589863003, '_wandb': {}, 'assignments': {}, 'metric': 'eval/loss'}
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--openai--whisper-small/snapshots/ecd562088fc463cdf07bf4a997944edeae03993e/config.json
Model config WhisperConfig {
  "_name_or_path": "openai/whisper-small",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "WhisperForCondition

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_steps' was locked by 'sweep' (ignored update).
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
300,1.247500,1.258242,62.154822


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
eval/wer,▁
train/epoch,▁▂▂▃▄▄▅▅▆▇▇███
train/global_step,▁▂▂▃▄▄▅▅▆▇▇███
train/learning_rate,▅█▇▇▆▅▅▄▃▂▂▁
train/loss,▃▅█▇▆▆▅▄▃▂▂▁
train/total_flos,▁
train/train_loss,▁


In [ ]:
best_trial

BestRun(run_id='xs50is5v', objective=24.97647280255976, hyperparameters={'learning_rate': 1.455023150831101e-05, 'warmup_steps': 244.41765045285135, 'weight_decay ': 0.006571702855352429, 'assignments': {}, 'metric': 'eval/loss'})

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    num_train_epochs=1,
    output_dir="whisper-small-se",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1.5e-5,
    warmup_steps=150,
    weight_decay=0.006,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=2000,
    logging_steps=100,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    hub_strategy="checkpoint",
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=None,
    model_init=model_init,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 12360
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
  Number of trainable parameters = 241734912
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.129600,0.280254,20.604447


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

TrainOutput(global_step=1000, training_loss=0.612234894990921, metrics={'train_runtime': 7837.9579, 'train_samples_per_second': 2.041, 'train_steps_per_second': 0.128, 'total_flos': 4.61505772191744e+18, 'train_loss': 0.612234894990921, 'epoch': 1.29})

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: sv-SE, split: test",
    "language": "sv-SE",
    "model_name": "Whisper Small Swedish",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard", # ?
}

In [ ]:
trainer.push_to_hub(**kwargs)

Saving model checkpoint to ./whisper-small-sv-SE
Configuration saved in ./whisper-small-sv-SE/config.json
Model weights saved in ./whisper-small-sv-SE/pytorch_model.bin
Feature extractor saved in ./whisper-small-sv-SE/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file last-checkpoint/optimizer.pt:   0%|          | 3.30k/1.80G [00:00<?, ?B/s]

Upload file runs/Dec06_14-05-42_95124521ccff/events.out.tfevents.1670335557.95124521ccff.74.2:  30%|###       …

Upload file pytorch_model.bin:   0%|          | 3.30k/922M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/ayberkuckun/whisper-small-sv-SE
   ab7fa22..37374c0  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/ayberkuckun/whisper-small-sv-SE
   ab7fa22..37374c0  main -> main

remote: ----------------------------------------------------------        
remote: Sorry, your push was rejected during YAML metadata verification:        
remote: - Error: "language[0]" must only contain lowercase characters        
remote: - Error: "language[0]" with value "sv-SE" is not valid. It must be an ISO 639-1, 639-2 or 639-3 code (two/three letters), or a special value like "code", "multilingual". If you want to use BCP-47 identifiers, you can specify them in language_bcp47.        
remote: - Error: "model-index[0].results[0].dataset.config" must be a string        
remote: ----------------------------------------------------------        
remote: Ple

'https://huggingface.co/ayberkuckun/whisper-small-sv-SE/commit/37374c00465683dc86270078f456665bda065ae4'